In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import numpy as np

In [2]:
steady_ratings = pd.read_csv('steady_ratings.csv')
ksteady = pd.read_csv('ksteady.csv', encoding='latin-1')
ks_ratings = pd.merge(ksteady,steady_ratings)

In [3]:
ks_ratings['song'] = ks_ratings['title'] +' - '+ ks_ratings['artist']
print(ks_ratings.shape)
ks_ratings.head()

(4403, 7)


,song_id,title,artist,tempo,user_id,rating,song
0,2,HANN (Alone),(G)I-DLE,104.971,12,4,HANN (Alone) - (G)I-DLE
1,2,HANN (Alone),(G)I-DLE,104.971,12,0,HANN (Alone) - (G)I-DLE
2,2,HANN (Alone),(G)I-DLE,104.971,22,5,HANN (Alone) - (G)I-DLE
3,2,HANN (Alone),(G)I-DLE,104.971,25,1,HANN (Alone) - (G)I-DLE
4,2,HANN (Alone),(G)I-DLE,104.971,27,3,HANN (Alone) - (G)I-DLE


In [4]:
numsteady_rating = ks_ratings.groupby('song')['rating'].count().reset_index()

In [5]:
numsteady_rating.head()

,song,rating
0,0X1=LOVESONG (I Know I Love You) feat. Seori -...,21
1,1 Billion Views - EXO-SC,18
2,1+1=0 (Feat. DEAN) - SURAN,24
3,10 Months - ENHYPEN,17
4,10 Out of 10 - 2PM,23


In [6]:
numsteady_rating.rename(columns={'rating':'no of ratings'}, inplace=True)
final_steady = ks_ratings.merge(numsteady_rating, on='song')
final_steady

,song_id,title,artist,tempo,user_id,rating,song,no of ratings
0,2,HANN (Alone),(G)I-DLE,104.971,12,4,HANN (Alone) - (G)I-DLE,23
1,2,HANN (Alone),(G)I-DLE,104.971,12,0,HANN (Alone) - (G)I-DLE,23
2,2,HANN (Alone),(G)I-DLE,104.971,22,5,HANN (Alone) - (G)I-DLE,23
3,2,HANN (Alone),(G)I-DLE,104.971,25,1,HANN (Alone) - (G)I-DLE,23
4,2,HANN (Alone),(G)I-DLE,104.971,27,3,HANN (Alone) - (G)I-DLE,23
...,...,...,...,...,...,...,...,...
4398,634,Eureka (Feat. Zion.T),ZICO,119.936,245,1,Eureka (Feat. Zion.T) - ZICO,13
4399,634,Eureka (Feat. Zion.T),ZICO,119.936,249,2,Eureka (Feat. Zion.T) - ZICO,13
4400,634,Eureka (Feat. Zion.T),ZICO,119.936,266,4,Eureka (Feat. Zion.T) - ZICO,13
4401,634,Eureka (Feat. Zion.T),ZICO,119.936,273,4,Eureka (Feat. Zion.T) - ZICO,13


In [7]:
final_steady = final_steady[final_steady['no of ratings']>=5] #only songs above 5 users have rated is qualified, to avoid noise
final_steady.shape

(4403, 8)

In [8]:
final_steady.drop_duplicates(['user_id','song'], inplace=True)
final_steady.shape

(4300, 8)

In [9]:
steady_pivot = final_steady.pivot_table(columns='user_id', index='song', values='rating')
steady_pivot.fillna(0, inplace=True)

In [10]:
print(steady_pivot.shape)
steady_pivot

(286, 332)


user_id,1,2,3,4,5,6,7,8,9,10,...,323,324,325,326,327,328,329,330,331,332
song,,,,,,,,,,,,,,,,,,,,,
0X1=LOVESONG (I Know I Love You) feat. Seori - TOMORROW X TOGETHER,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1 Billion Views - EXO-SC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1+1=0 (Feat. DEAN) - SURAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Months - ENHYPEN,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
10 Out of 10 - 2PM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eight(Prod.&Feat. SUGA of BTS) - IU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gogobebe - MAMAMOO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
mumumumuch - MAMAMOO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from scipy.sparse import csr_matrix
steady_sparse = csr_matrix(steady_pivot)
type(steady_sparse)

scipy.sparse.csr.csr_matrix

In [12]:
from sklearn.neighbors import NearestNeighbors
steady_model = NearestNeighbors(algorithm='brute')

In [13]:
steady_model.fit(steady_sparse)

NearestNeighbors(algorithm='brute')

In [14]:
distances, suggestions = steady_model.kneighbors(steady_pivot.iloc[4,:].values.reshape(1,-1), n_neighbors=15)

In [15]:
suggestions

array([[  4,  62,  19, 279,  54, 198, 172, 272,  18, 274, 283, 229, 273,
        192, 201]], dtype=int64)

In [16]:
for i in range(len(suggestions)):
    print(steady_pivot.index[suggestions[i]])

Index(['10 Out of 10 - 2PM', 'Count 1, 2 - TOO', 'Anpanman - BTS',
       'Zombie - PURPLE KISS', 'Cause I'm God Girl - HyunA',
       'Palette (feat. G-DRAGON) - IU', 'Never Ever - GOT7',
       'Wind flower - MAMAMOO', 'Am I Wrong - BTS', 'Woo Hoo - MAMAMOO',
       'mumumumuch - MAMAMOO', 'Senorita - VAV', 'With You - ASTRO',
       'Overdose - EXO-K', 'Piano Man 2021 - MAMAMOO'],
      dtype='object', name='song')


In [17]:
steady_pivot.index[50]

'CHECKMATE - Various Artists'

In [18]:
np.where(steady_pivot.index=='CHECKMATE - Various Artists')[0][0]

50

In [19]:
def recommend_kmusic_steady(song):
    song_id=np.where(steady_pivot.index==song)[0][0]
    distances, suggestions = steady_model.kneighbors(steady_pivot.iloc[song_id,:].values.reshape(1,-1), n_neighbors=15)
    
    for i in range(len(suggestions)):
        print(steady_pivot.index[suggestions[i]])

In [21]:
recommend_kmusic_steady('Woo Hoo - MAMAMOO')

Index(['Woo Hoo - MAMAMOO', 'Count 1, 2 - TOO',
       'Palette (feat. G-DRAGON) - IU', 'Cause I'm God Girl - HyunA',
       'Zombie - PURPLE KISS', 'Am I Wrong - BTS', 'Anpanman - BTS',
       'Senorita - VAV', 'Overdose - EXO-K', 'Crazy Sexy Cool - ASTRO',
       'With You - ASTRO', 'Monster - Red Velvet - IRENE & SEULGI',
       'Wind flower - MAMAMOO', 'Ah Yeah - EXID', 'HOME;RUN - SEVENTEEN'],
      dtype='object', name='song')


In [22]:
ratesteady = ks_ratings.groupby('song_id')['rating'].count().reset_index()
ratesteady

,song_id,rating
0,2,23
1,3,21
2,4,16
3,5,11
4,6,11
...,...,...
281,625,19
282,628,17
283,629,14
284,631,14


In [23]:
steady_list = pd.merge(ksteady,ratesteady)
steady_list.rename(columns={'rating':'no of ratings'}, inplace=True)
steady_list 

,song_id,title,artist,tempo,no of ratings
0,2,HANN (Alone),(G)I-DLE,104.971,23
1,3,LATATA,(G)I-DLE,98.100,21
2,4,LION,(G)I-DLE,109.989,16
3,5,Oh my god,(G)I-DLE,109.970,11
4,6,Senorita,(G)I-DLE,120.021,11
...,...,...,...,...,...
281,625,REALLY REALLY,WINNER,103.006,19
282,628,Open Mind,WONHO,110.043,17
283,629,FACE,Woosung,113.034,14
284,631,Do Better,Y teen (MONSTA X & WJSN),101.012,14


In [24]:
steady_list['song'] = steady_list['title'] +' - '+ steady_list['artist']
steady_list 

,song_id,title,artist,tempo,no of ratings,song
0,2,HANN (Alone),(G)I-DLE,104.971,23,HANN (Alone) - (G)I-DLE
1,3,LATATA,(G)I-DLE,98.100,21,LATATA - (G)I-DLE
2,4,LION,(G)I-DLE,109.989,16,LION - (G)I-DLE
3,5,Oh my god,(G)I-DLE,109.970,11,Oh my god - (G)I-DLE
4,6,Senorita,(G)I-DLE,120.021,11,Senorita - (G)I-DLE
...,...,...,...,...,...,...
281,625,REALLY REALLY,WINNER,103.006,19,REALLY REALLY - WINNER
282,628,Open Mind,WONHO,110.043,17,Open Mind - WONHO
283,629,FACE,Woosung,113.034,14,FACE - Woosung
284,631,Do Better,Y teen (MONSTA X & WJSN),101.012,14,Do Better - Y teen (MONSTA X & WJSN)


In [26]:
import pickle

In [27]:
pickle.dump(steady_list,open('ksteady.pkl','wb'))

In [29]:
steady_list['song']

0                   HANN (Alone) - (G)I-DLE
1                         LATATA - (G)I-DLE
2                           LION - (G)I-DLE
3                      Oh my god - (G)I-DLE
4                       Senorita - (G)I-DLE
                       ...                 
281                  REALLY REALLY - WINNER
282                       Open Mind - WONHO
283                          FACE - Woosung
284    Do Better - Y teen (MONSTA X & WJSN)
285            Eureka (Feat. Zion.T) - ZICO
Name: song, Length: 286, dtype: object

In [30]:
steady_list.to_dict()

{'song_id': {0: 2,
  1: 3,
  2: 4,
  3: 5,
  4: 6,
  5: 7,
  6: 10,
  7: 11,
  8: 21,
  9: 24,
  10: 26,
  11: 31,
  12: 34,
  13: 36,
  14: 37,
  15: 42,
  16: 43,
  17: 44,
  18: 45,
  19: 46,
  20: 47,
  21: 51,
  22: 52,
  23: 53,
  24: 54,
  25: 62,
  26: 63,
  27: 65,
  28: 66,
  29: 68,
  30: 74,
  31: 77,
  32: 95,
  33: 98,
  34: 102,
  35: 105,
  36: 108,
  37: 110,
  38: 113,
  39: 115,
  40: 117,
  41: 119,
  42: 120,
  43: 122,
  44: 123,
  45: 126,
  46: 128,
  47: 129,
  48: 130,
  49: 131,
  50: 132,
  51: 134,
  52: 137,
  53: 142,
  54: 145,
  55: 146,
  56: 153,
  57: 156,
  58: 157,
  59: 160,
  60: 161,
  61: 162,
  62: 163,
  63: 165,
  64: 167,
  65: 172,
  66: 176,
  67: 178,
  68: 180,
  69: 184,
  70: 185,
  71: 193,
  72: 194,
  73: 196,
  74: 197,
  75: 198,
  76: 201,
  77: 203,
  78: 204,
  79: 205,
  80: 206,
  81: 207,
  82: 208,
  83: 211,
  84: 212,
  85: 213,
  86: 218,
  87: 221,
  88: 222,
  89: 223,
  90: 224,
  91: 225,
  92: 229,
  93: 230,
  94:

In [31]:
pickle.dump(steady_list.to_dict(),open('ksteady_dict.pkl','wb'))

In [32]:
pickle.dump(steady_pivot,open('steady_pivot.pkl','wb'))

In [33]:
pickle.dump(steady_model,open('steady_model.pkl','wb'))

In [34]:
steady_list['tempo']

0      104.971
1       98.100
2      109.989
3      109.970
4      120.021
        ...   
281    103.006
282    110.043
283    113.034
284    101.012
285    119.936
Name: tempo, Length: 286, dtype: float64